# Convolution Image Net

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_visualization import variable_summaries
from utils.data import init_model_logging
from utils.nn_graph import simple_layer
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

## Load data

In [ ]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
img_shape = (28, 28)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

In [ ]:
def data_generator():
    while True:
        images, labels = data.train.next_batch(32)
        images = images.reshape((-1, 28, 28, 1))
        yield (images, labels)
    

## Keras

In [ ]:
import keras as k
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense
from keras.models import Model

In [ ]:
images = Input(shape=(28, 28, 1))
conv_layer_1 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', 
                      kernel_initializer='glorot_uniform')(images)
conv_layer_1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_layer_1)

conv_layer_2 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(conv_layer_1)
conv_layer_2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_layer_2)

flatten_layer = Flatten()(conv_layer_2)
prediction = Dense(10, activation='softmax')(flatten_layer)

model = Model(inputs=images, outputs=prediction)

In [ ]:
conv_layer_1._keras_shape, conv_layer_2._keras_shape

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mse', 'accuracy'])

In [ ]:
valid_images = data.validation.images.reshape((-1, 28, 28, 1))
valid_labels = data.validation.labels

In [ ]:
valid_images.shape, valid_labels.shape

In [ ]:
model.fit_generator(data_generator(), steps_per_epoch=100, epochs=10,
          validation_data=(valid_images, valid_labels))

In [ ]:
_prediction = model.predict(valid_images)

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
fp = get_false_positives(df, 'Shirt')

In [ ]:
plot_examples(fp)

In [ ]:
plot_coocurance_matrix(df, use_top3=False, use_log=False)